In [12]:
!pip install langchain faiss-cpu sentence-transformers huggingface-hub transformers langchain-community

  Using cached langchain-1.2.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached faiss_cpu-1.13.1-cp311-cp311-win_amd64.whl.metadata (7.6 kB)
  Using cached sentence_transformers-5.2.0-py3-none-any.whl.metadata (16 kB)
  Using cached huggingface_hub-1.2.3-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_core-1.2.4-py3-none-any.whl.metadata (3.7 kB)
  Using cached langgraph-1.0.5-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached torch-2.9.1-cp311-cp311-win_amd64.whl.metadata (30 kB)
  Using cached scikit_learn-1.8.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl.metadata (5.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cach


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import os
os.makedirs("data", exist_ok=True)

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("test.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
len(chunks)

d:\final Year project\Research_backend\RAG\venvRAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db = FAISS.from_documents(chunks, embeddings)
retriever = db.as_retriever()


C:\Users\mathivathanan\AppData\Local\Temp\ipykernel_31832\567402945.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline(
    "text-generation",
    model="Qwen/Qwen2.5-1.5B-Instruct",
    max_new_tokens=256,
    temperature=0.2
)

llm = HuggingFacePipeline(pipeline=pipe)


d:\final Year project\Research_backend\RAG\venvRAG\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mathivathanan\.cache\huggingface\hub\models--Qwen--Qwen2.5-1.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fa

In [ ]:
def rag_chain(question):
    # 1. Retrieve similar docs
    docs = retriever.invoke(question)

    # 2. Combine the doc text
    context = "\n\n".join([d.page_content for d in docs])

    # 3. Format the prompt
    prompt_text = f"""
You are an AI assistant. Use ONLY the provided context to answer.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question: {question}

Answer:
"""

    # 4. Call the model
    raw = llm.invoke(prompt_text)
    answer = raw.replace(prompt_text, "").strip()
    return answer

In [ ]:
response = rag_chain("What is Karthik's Show?")
print(response)

I don't know. The given context does not provide any information about a show called "Karthik's Show". It only mentions artificial intelligence and its applications in various fields like healthcare, finance, education, and transportation. There is no mention of a specific TV show named after someone named Karthik. Therefore, based solely on this context, I cannot determine what "Karthik's Show" might be referring to. To accurately answer this question, additional information or clarification would be needed.
